In [8]:
import keras_tuner
from tensorflow import keras
import tensorflow as tf
import os
import cv2
import numpy as np
from keras import Sequential
from keras.models import Model
from keras.layers import Dense, Flatten, Input, Concatenate
import matplotlib.pyplot as plt
import tqdm
from sklearn.preprocessing import LabelBinarizer
import pathlib
from keras.applications.densenet import DenseNet121
from keras.layers import BatchNormalization, Activation, Dropout, MaxPooling2D,Conv2D
import pandas as pd

In [9]:
size = 96

In [10]:
#EmotioNet
EmotioNet_FACS_X_train = pd.read_pickle("./Clean_datasets/EmotioNet/EmotioNet_FACS_X_train")
EmotioNet_FACS_y_train = pd.read_pickle("./Clean_datasets/EmotioNet/EmotioNet_FACS_y_train")
EmotioNet_FACS_X_val = pd.read_pickle("./Clean_datasets/EmotioNet/EmotioNet_FACS_X_val")
EmotioNet_FACS_y_val = pd.read_pickle("./Clean_datasets/EmotioNet/EmotioNet_FACS_y_val")
X_train = EmotioNet_FACS_X_train
X_val = EmotioNet_FACS_X_val
y_train = EmotioNet_FACS_y_train
y_val = EmotioNet_FACS_y_val

In [23]:
y_train = EmotioNet_FACS_y_train
y_val = EmotioNet_FACS_y_val

In [11]:
from skimage.transform import resize
def resize_images(images):
    resized_images = []
    for image in images:
        resized_images.append(resize(image, (size, size)))
    return np.asarray(resized_images).astype('float32')

In [12]:
X_train = resize_images(X_train)
X_val = resize_images(X_val)
#X_test = resize_images(X_val)

In [13]:
X_train = np.resize(X_train,(len(X_train),size,size))
X_val = np.resize(X_val,(len(X_val),size,size))

In [14]:
X_train = np.resize(X_train,(len(X_train),size,size,1))
X_val = np.resize(X_val,(len(X_val),size,size,1))
#X_test = np.resize(X_test,(len(X_test),size,size,1))

In [15]:
print(np.shape(X_train))
print(np.shape(X_val))
#print(np.shape(X_test))

(12629, 96, 96, 1)
(1578, 96, 96, 1)


In [16]:
y_train

,1,2,4,5,6,9,10,12,15,17,18,20,24,25,26,28
0,0,0,0,1,0,0,1,1,1,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
2,1,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15886,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
15888,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
15890,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
15893,1,1,0,1,0,0,0,0,0,0,1,0,0,1,1,0


In [24]:
np.shape(y_val)

(1578, 16)

In [25]:
np.shape(y_train)

(12629, 16)

In [26]:
NUM_EPOCHS = 20

In [27]:
def change_input_layer_dense(pretrained_model, first_conv, h, w, c):
    pretrained_config = pretrained_model.get_config()

    # Change the input shape from the model
    pretrained_config["layers"][0]["config"]["batch_input_shape"] = (None, h, w, c)

    updated_model = Model.from_config(pretrained_config)


    def avg_weights(weights):
        average_weights = np.mean(weights, axis=-2).reshape(weights[:, :, -1:, :].shape)
        return(average_weights)

    pretrained_updated_config = updated_model.get_config()
    pretrained_updated_layer_names = [pretrained_updated_config['layers'][x]['name'] for x in range(len(pretrained_updated_config['layers']))]
    print(pretrained_updated_layer_names)
    first_conv_name = pretrained_updated_layer_names[first_conv]

    for layer in pretrained_model.layers:
        if layer.name in pretrained_updated_layer_names:
            if layer.get_weights() != []:  #All convolutional layers and layers with weights (no input layer or any pool layers)
                target_layer = updated_model.get_layer(layer.name)
            
                if layer.name in first_conv_name:    #For the first convolutionl layer
                    weights = layer.get_weights()[0]
                    
                    weights_single_channel = avg_weights(weights)
                                                                
                    target_layer.set_weights([weights_single_channel])  #Now set weights for the first conv. layer
                    target_layer.trainable = False   #You can make this trainable if you want. 
            
                else:
                    target_layer.set_weights(layer.get_weights())   #Set weights to all other layers. 
                    target_layer.trainable = True #You can make this trainable if you want.
    updated_model.summary()
    return updated_model

In [28]:
densenet_model = DenseNet121(include_top=False, weights='imagenet')
densenet_updated = change_input_layer_dense(densenet_model, 2, size, size, 1)

['input_2', 'zero_padding2d_2', 'conv1/conv', 'conv1/bn', 'conv1/relu', 'zero_padding2d_3', 'pool1', 'conv2_block1_0_bn', 'conv2_block1_0_relu', 'conv2_block1_1_conv', 'conv2_block1_1_bn', 'conv2_block1_1_relu', 'conv2_block1_2_conv', 'conv2_block1_concat', 'conv2_block2_0_bn', 'conv2_block2_0_relu', 'conv2_block2_1_conv', 'conv2_block2_1_bn', 'conv2_block2_1_relu', 'conv2_block2_2_conv', 'conv2_block2_concat', 'conv2_block3_0_bn', 'conv2_block3_0_relu', 'conv2_block3_1_conv', 'conv2_block3_1_bn', 'conv2_block3_1_relu', 'conv2_block3_2_conv', 'conv2_block3_concat', 'conv2_block4_0_bn', 'conv2_block4_0_relu', 'conv2_block4_1_conv', 'conv2_block4_1_bn', 'conv2_block4_1_relu', 'conv2_block4_2_conv', 'conv2_block4_concat', 'conv2_block5_0_bn', 'conv2_block5_0_relu', 'conv2_block5_1_conv', 'conv2_block5_1_bn', 'conv2_block5_1_relu', 'conv2_block5_2_conv', 'conv2_block5_concat', 'conv2_block6_0_bn', 'conv2_block6_0_relu', 'conv2_block6_1_conv', 'conv2_block6_1_bn', 'conv2_block6_1_relu', 'co

In [29]:
num_classes = 16
NUM_EPOCHS = 20

In [30]:
precision = keras.metrics.Precision(name="Precision")
recall = keras.metrics.Recall(name="Recall")
regularizer = tf.keras.regularizers.l1_l2(0, 0.001)

In [31]:
from keras import backend as K

def custom_f1(y_true, y_pred):
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))

        recall = TP / (Positives+K.epsilon())
        return recall


    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))

        precision = TP / (Pred_Positives+K.epsilon())
        return precision

    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)

    return 2*((precision*recall)/(precision+recall+K.epsilon()))

auc_metric = tf.keras.metrics.AUC(
num_thresholds=200, curve="ROC",
summation_method="interpolation",
thresholds=None, multi_label=False
)


In [32]:
num_epoch_scheduler = 10
def scheduler(epoch, lr):
    if epoch < num_epoch_scheduler:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

In [34]:
import datetime
folder_name = 'logs/DenseNet/Fine-Tuning/Emotion/'
log_folder = folder_name + str(num_epoch_scheduler) + "tf.math.exp(-0.1)"


In [35]:
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(filepath='./chkpts/Fine-Tuning/DenseNet',monitor='val_custom_f1', mode='max',save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='max',patience=2)
#The callbacks list now becomes:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_folder)
scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)
callbacks=[scheduler, tensorboard_callback]

In [36]:
model = Sequential()
model.add(densenet_updated)
model.add(Flatten())
model.add(Dense(1024, kernel_regularizer=regularizer))
model.add(BatchNormalization(scale=False,
                                center=False))
model.add(Activation('relu'))
model.add(Dropout(rate=0.4))
model.add(Dense(units=num_classes, activation="softmax",kernel_regularizer=regularizer)) 
model.compile(optimizer=tf.keras.optimizers.Adam(),
            loss='binary_crossentropy', metrics=[precision,recall,custom_f1, auc_metric,'accuracy'])

In [37]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=NUM_EPOCHS,
callbacks=callbacks)

Epoch 1/20
 74/395 [====>.........................] - ETA: 16:47 - loss: 1.1999 - Precision: 0.8262 - Recall: 0.1502 - custom_f1: 0.2523 - auc: 0.8367 - accuracy: 0.2293

KeyboardInterrupt: 

In [ ]:
def build_model(hp):
    lrate = hp.Float('lrate', 1e-4, 1e-1, sampling='log')
    l1 = 0
    l2 = hp.Choice('l2',  values=[0.0, 1e-1, 1e-2, 1e-3, 1e-4])
    num_hidden = hp.Int('num_hidden', 32,1024,32)
    regularizer = tf.keras.regularizers.l1_l2(l1, l2)
    model = Sequential()
    model.add(densenet_updated)
    model.add(Flatten())
    model.add(Dense(num_hidden, kernel_regularizer=regularizer))
    model.add(BatchNormalization(scale=False,
                                    center=False))
    model.add(Activation('relu'))
    model.add(Dropout(rate=0.4))
    model.add(Dense(units=num_classes, activation="softmax",kernel_regularizer=regularizer)) 
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lrate),
                  loss='categorical_crossentropy', metrics=[precision,recall,custom_f1, auc_metric,'accuracy'])
    return model


In [ ]:
tuner = keras_tuner.BayesianOptimization(
    build_model,
    objective=keras_tuner.Objective('val_custom_f1', 'max'),
    max_trials = 10,
    overwrite=False
)

In [ ]:
tuner.search(
    X_train, y_train, validation_data=(X_val, y_val),
    epochs=1
)

Trial 1 Complete [00h 00m 29s]
val_custom_f1: 0.6180554628372192

Best val_custom_f1 So Far: 0.6180554628372192
Total elapsed time: 00h 00m 29s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
0.00053772        |0.0044104         |lrate
0.0001            |0                 |l2
640               |512               |num_hidden

12/28 [===========>..................] - ETA: 12s - loss: 2.4039 - Precision: 0.5449 - Recall: 0.3684 - custom_f1: 0.3597 - auc: 0.7177 - accuracy: 0.4193

KeyboardInterrupt: 

In [ ]:
topN = 2
for x in range(topN):
print(tuner.get_best_hyperparameters(topN)[x].values)
print(tuner.get_best_models(topN)[x].summary())